In [ ]:
# link one https://www.gracedupage.org/about-us 

In [3]:
from haystack.nodes import LinkContentFetcher
from haystack.schema import Document
from typing import List 
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import AnswerParser, PromptNode, PromptTemplate
from haystack import Pipeline
from haystack.nodes import  DensePassageRetriever
import os
from dotenv import load_dotenv

/Users/macpro/anaconda3/envs/llm-pipelines/lib/python3.10/site-packages/quantulum3/classifier.py:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/Users/macpro/anaconda3/envs/llm-pipelines/lib/python3.10/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/Users/macpro/anaconda3/envs/llm-pipelines/lib/python3.10/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.logging')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setupto

In [4]:
def initialize_rag_pipeline(retriever, openai_key):
    """
    Initialize a pipeline for RAG-based question answering.

    Args:
        retriever (DensePassageRetriever): Dense passage retriever.
        openai_key (str): API key for OpenAI.

    Returns:
        query_pipeline (Pipeline): Pipeline for RAG-based question answering.
    """
    prompt_template = PromptTemplate(prompt = """"Answer the following query based on the provided context. \
                                                Please include the relevant link or links that you used to answer the question. \
                                                If the context does
                                                not include an answer, reply with 'The data does not contain information related to the question'.\n
                                                Query: {query}\n
                                                Documents: {join(documents)}
                                                Answer: 
                                            """,
                                            output_parser=AnswerParser())
    prompt_node = PromptNode(model_name_or_path = "gpt-4",
                            api_key = openai_key,
                            default_prompt_template = prompt_template,
                            max_length = 500,
                            model_kwargs={"stream":True})

    query_pipeline = Pipeline()
    query_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
    query_pipeline.add_node(component=prompt_node, name="PromptNode", inputs=["Retriever"])

    return query_pipeline

In [5]:
load_dotenv("../.env")
openai_key = os.getenv("OPENAI_API_KEY")


In [6]:
# Initialize document store
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

retriever = DensePassageRetriever(
            document_store=document_store,
            query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
            passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
            use_gpu=True,
            embed_title=True,
)

/Users/macpro/anaconda3/envs/llm-pipelines/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
link_content_fetcher = LinkContentFetcher()
documents = []
links = ["https://www.gracedupage.org/about-us",
         'https://www.gracedupage.org/who-is-jesus',
         'https://www.gracedupage.org/purpose',
         'https://www.gracedupage.org/history',
         'https://www.gracedupage.org/pastoral-ministries',
         'https://www.gracedupage.org/leadership',
         'https://www.gracedupage.org/baptism-and-membership'
         ]

for link in links:
    documents : List[Document] = link_content_fetcher.fetch(url=link)
    document_store.write_documents(documents)



Writing Documents: 10000it [00:00, 13883.84it/s]        
Writing Documents: 10000it [00:00, 19921.28it/s]        
Writing Documents: 10000it [00:00, 100371.26it/s]       
Writing Documents: 10000it [00:00, 527625.23it/s]       
Writing Documents: 10000it [00:00, 680219.91it/s]       
Writing Documents: 10000it [00:00, 225250.74it/s]       
Writing Documents: 10000it [00:00, 291400.62it/s]       


In [8]:
 # Add documents embeddings to index
document_store.update_embeddings(retriever=retriever)

Documents Processed: 10000 docs [00:20, 481.58 docs/s]       


In [9]:
# Initialize pipeline
query_pipeline = initialize_rag_pipeline(retriever=retriever, openai_key=openai_key)


In [10]:
answer = query_pipeline.run(query="What is the purpose of Grace Church?", documents=documents)

The purpose of Grace Church is "to magnify the worth of God by exalting the Savior, equipping the saints and extending Christ's kingdom." This includes various activities such as corporate worship, small gatherings, in-church service, and outreach. The church also emphasizes the importance of preaching the truth and authority of God's Word, praying according to God's Word led by His Holy Spirit, proclaiming the life-changing message of the gospel, shepherding individuals and families toward maturity in Jesus Christ, resting in the completeness of God's sovereignty, and doing all to the glory of God.

In [ ]:
answer['answers'][0].answer

In [ ]:
answer['answers']

[<Answer {'answer': 'The purpose of Grace Church is "to magnify the worth of God by exalting the Savior, equipping the saints and extending Christ\'s kingdom." This includes various activities such as corporate worship, small gatherings, in-church service, and outreach. The church also emphasizes the importance of preaching the truth and authority of God\'s Word, praying according to God\'s Word led by His Holy Spirit, proclaiming the life-changing message of the gospel, shepherding individuals and families toward maturity in Jesus Christ, resting in the completeness of God\'s sovereignty, and doing all to the glory of God.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['387fbf3c6f3e17c261aa08b41b5a1f4a', 'c76ee9bcae4bb2c2bd47296fefa0ad4a', 'aa16e70e066c17c15778ef504d0ddd57', 'ab8b274f1e6a53540a235245fc1dee6b', '491a57b18a37177e7d064a6af5734332', 'e9e7295e9ca8b32b8bf9be740783fa9c', '6f5f29dec5f58b59c75f1

In [13]:
answer.keys()

dict_keys(['answers', 'invocation_context', 'documents', 'root_node', 'params', 'query', 'node_id'])

In [16]:
answer

{'answers': [<Answer {'answer': 'The purpose of Grace Church is "to magnify the worth of God by exalting the Savior, equipping the saints and extending Christ\'s kingdom." This includes various activities such as corporate worship, small gatherings, in-church service, and outreach. The church also emphasizes the importance of preaching the truth and authority of God\'s Word, praying according to God\'s Word led by His Holy Spirit, proclaiming the life-changing message of the gospel, shepherding individuals and families toward maturity in Jesus Christ, resting in the completeness of God\'s sovereignty, and doing all to the glory of God.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['387fbf3c6f3e17c261aa08b41b5a1f4a', 'c76ee9bcae4bb2c2bd47296fefa0ad4a', 'aa16e70e066c17c15778ef504d0ddd57', 'ab8b274f1e6a53540a235245fc1dee6b', '491a57b18a37177e7d064a6af5734332', 'e9e7295e9ca8b32b8bf9be740783fa9c', '6f5f29dec